## Follows Create_peak_by_tissue_and_Cluster

In [2]:
import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
sns.set_context('notebook')
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
#rcParams['font.family'] = 'sans-serif'
#rcParams['font.sans-serif'] = ['Tahoma']
import tqdm
### notebook specific configuration ###
from os.path import basename
mpl.style.use('fivethirtyeight')

from Modules.Homer import *

sys.path.append('/home/isshamie/software/NGS-Pipeline')

data_folder = '/data/isshamie/CH_tissue_TSS/'

ref_fa = '/data/genome/hamster/picr/picr.fa'
annotation = '/data/genome/hamster/picr/updated_final_sort.gff3'
tss_annotation = '/data/isshamie/genome/start_site_mRNA_updated_final_sort.tsv'
mrna_peak = '/data/isshamie/TSS_CHO/mRNA.peak'

tissues = ['BloondnegSpleen','BMDM1hKLA1','BMDMwt','Brain','Brain7neg1',
          'FemaleReproductive','Heart','Intestine','Kidney','Liver','Lung',
           'MiscOrgans','Muscle','Pancreas','Skin','Spleen','Total','CHO']

len(tissues)

### Load in meta_sampe

meta_samples = pd.read_csv('Results/meta_samples.csv',index_col=0)

long_meta_samples = pd.read_csv('Results/long_meta_samples.csv',index_col=0)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from tqdm import tqdm_notebook

In [3]:
peak_tissue_matrix = pd.read_csv('Results/merged_samples/duplicatesSeparate_peaks_merged.tsv',index_col=0,sep='\t')

/home/isshamie/software/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
### First filter peaks to only promoter
anno_peaks = pd.read_csv('Results/merged_samples/duplicateSeparate_peaks_merged_anno.tsv',sep='\t',index_col=0)
anno_peaks.dropna(axis=1,how='all',inplace=True)
anno_peaks = anno_peaks.fillna('')
promoter_peaks = anno_peaks[anno_peaks['Annotation'].str.contains('promoter')]
promoter_peaks_index = promoter_peaks.index
promoter_tissue_matrix = peak_tissue_matrix[peak_tissue_matrix.index.isin(promoter_peaks_index)]

In [5]:
print 'Number of promoter peaks: ',len(promoter_peaks)
print 'Number of Unique annotations: ' , len(promoter_peaks.groupby('Annotation'))

Number of promoter peaks:  135262
Number of Unique annotations:  82079


## Load in the unique TSS gff file

In [4]:
tss = pd.read_csv(tss_annotation,sep='\t',index_col=0)

In [7]:
tss.head()

,Chr,Start,End,Strand,ID,Parent,gene_id,gene,transcript_id,cs,Length
ID=asmbl_2;Parent=gene_1_2;gene_id=gene_1_2;gene=SERHL;transcript_id=asmbl_2;cs=2,picr_0,176,38694,+,asmbl_2,gene_1_2,gene_1_2,SERHL,asmbl_2,2,38518
ID=asmbl_4;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_4;cs=1,picr_0,7715,46630,-,asmbl_4,gene_2,gene_2,RRP7A,asmbl_4,1,38915
ID=asmbl_7;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_7;cs=1,picr_0,7759,44597,-,asmbl_7,gene_2,gene_2,RRP7A,asmbl_7,1,36838
ID=asmbl_8;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_8;cs=1,picr_0,7767,44597,-,asmbl_8,gene_2,gene_2,RRP7A,asmbl_8,1,36830
ID=asmbl_9;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_9;cs=1,picr_0,7789,44568,-,asmbl_9,gene_2,gene_2,RRP7A,asmbl_9,1,36779


## Take peaks +100 to -1kb
### Create additional column with the nearest TSS 
### Groupby that column, histogram of how many peaks per TSS
#### Also determine a) average distance between start sites b) scatter plot of top 2 highest, if only one, color that one but just duplicate it


In [ ]:
anno_peaks['Nearest TSS'] = ''
anno_peaks['Distance to TSS'] = np.infty
anno_peaks['isSameStrand'] = False

for ind,val in tqdm.tqdm(anno_peaks.iterrows()):
    filt = tss[val['Chr'] == tss['Chr']]
    peak_start = (val['Start']+val['End'])/2
    if len(filt) != 0:
        filt2 = np.abs(filt['Start'] - peak_start).argmin()
        anno_peaks.set_value(ind,'Nearest TSS',tss.loc[filt2,'transcript_id'])
        anno_peaks.set_value(ind,'Distance to TSS',peak_start - tss.loc[filt2,'Start'])
        anno_peaks.set_value(ind,'isSameStrand',val['Strand'] == tss.loc[filt2,'Strand'])



0it [00:00, ?it/s]
1it [00:00,  1.72it/s]

9it [00:00,  2.44it/s]
24it [00:00,  3.46it/s]
39it [00:00,  4.89it/s]
54it [00:00,  6.88it/s]
69it [00:01,  9.64it/s]
84it [00:01, 13.39it/s]
99it [00:01, 18.40it/s]
113it [00:01, 24.87it/s]
128it [00:01, 33.08it/s]
143it [00:01, 43.04it/s]
158it [00:01, 54.62it/s]
173it [00:01, 66.85it/s]
188it [00:01, 79.46it/s]
203it [00:02, 91.12it/s]
218it [00:02, 102.11it/s]
233it [00:02, 112.06it/s]
248it [00:02, 118.86it/s]
263it [00:02, 124.87it/s]
278it [00:02, 129.69it/s]
293it [00:02, 132.96it/s]
308it [00:02, 134.59it/s]
323it [00:02, 136.72it/s]
338it [00:02, 138.28it/s]
353it [00:03, 138.66it/s]
368it [00:03, 137.96it/s]
382it [00:03, 137.25it/s]
397it [00:03, 138.28it/s]
411it [00:03, 138.17it/s]
426it [00:03, 138.62it/s]
441it [00:03, 139.53it/s]
456it [00:03, 139.92it/s]
471it [00:03, 139.69it/s]
485it [00:04, 137.26it/s]
499it [00:04, 136.92it/s]
513it [00:04, 137.52it/s]
528it [00:04, 139.19it/s]
543it [00:04, 139.47it/s]
558it [00:04, 14

8071it [00:58, 140.45it/s]
8086it [00:58, 141.37it/s]
8101it [00:58, 141.65it/s]
8116it [00:58, 141.01it/s]
8131it [00:58, 132.72it/s]
8145it [00:58, 134.22it/s]
8159it [00:59, 135.18it/s]
8173it [00:59, 135.66it/s]
8188it [00:59, 137.72it/s]
8202it [00:59, 138.07it/s]
8216it [00:59, 138.21it/s]
8231it [00:59, 139.68it/s]
8246it [00:59, 140.78it/s]
8261it [00:59, 141.11it/s]
8276it [00:59, 141.50it/s]
8291it [00:59, 142.25it/s]
8306it [01:00, 143.24it/s]
8321it [01:00, 144.42it/s]
8336it [01:00, 143.19it/s]
8351it [01:00, 143.45it/s]
8366it [01:00, 143.99it/s]
8381it [01:00, 143.32it/s]
8396it [01:00, 142.64it/s]
8411it [01:00, 141.82it/s]
8426it [01:00, 143.35it/s]
8441it [01:01, 143.65it/s]
8456it [01:01, 142.13it/s]
8471it [01:01, 140.56it/s]
8486it [01:01, 139.75it/s]
8501it [01:01, 140.59it/s]
8516it [01:01, 141.00it/s]
8531it [01:01, 139.63it/s]
8546it [01:01, 140.11it/s]
8561it [01:01, 139.83it/s]
8576it [01:01, 141.10it/s]
8591it [01:02, 141.22it/s]
8606it [01:02, 140.53it/s]
8

15892it [01:54, 139.83it/s]
15907it [01:54, 141.43it/s]
15922it [01:54, 142.62it/s]
15937it [01:54, 131.87it/s]
15952it [01:54, 135.04it/s]
15967it [01:55, 136.89it/s]
15982it [01:55, 139.29it/s]
15997it [01:55, 140.22it/s]
16012it [01:55, 141.38it/s]
16027it [01:55, 142.83it/s]
16042it [01:55, 142.60it/s]
16057it [01:55, 142.46it/s]
16072it [01:55, 141.78it/s]
16087it [01:55, 141.91it/s]
16102it [01:55, 142.20it/s]
16117it [01:56, 143.16it/s]
16132it [01:56, 143.10it/s]
16147it [01:56, 142.96it/s]
16162it [01:56, 142.23it/s]
16177it [01:56, 142.16it/s]
16192it [01:56, 142.33it/s]
16207it [01:56, 142.59it/s]
16222it [01:56, 143.20it/s]
16237it [01:56, 143.34it/s]
16252it [01:57, 143.35it/s]
16267it [01:57, 141.76it/s]
16282it [01:57, 140.62it/s]
16297it [01:57, 142.37it/s]
16312it [01:57, 142.83it/s]
16327it [01:57, 143.85it/s]
16342it [01:57, 144.12it/s]
16357it [01:57, 144.05it/s]
16372it [01:57, 143.12it/s]
16387it [01:57, 143.40it/s]
16402it [01:58, 142.97it/s]
16417it [01:58, 140.

23691it [02:50, 138.18it/s]
23705it [02:50, 137.56it/s]
23720it [02:50, 139.27it/s]
23735it [02:50, 140.55it/s]
23750it [02:50, 139.33it/s]
23765it [02:50, 140.71it/s]
23780it [02:50, 142.61it/s]
23795it [02:50, 142.81it/s]
23810it [02:50, 143.01it/s]
23825it [02:51, 143.02it/s]
23840it [02:51, 142.97it/s]
23855it [02:51, 142.97it/s]
23870it [02:51, 142.71it/s]
23885it [02:51, 142.46it/s]
23900it [02:51, 141.02it/s]
23915it [02:51, 139.26it/s]
23929it [02:51, 139.08it/s]
23943it [02:51, 138.42it/s]
23957it [02:52, 137.11it/s]
23972it [02:52, 138.20it/s]
23987it [02:52, 139.85it/s]
24001it [02:52, 139.07it/s]
24015it [02:52, 138.55it/s]
24029it [02:52, 138.34it/s]
24044it [02:52, 139.80it/s]
24059it [02:52, 140.19it/s]
24074it [02:52, 141.39it/s]
24089it [02:52, 141.47it/s]
24104it [02:53, 140.97it/s]
24119it [02:53, 140.61it/s]
24134it [02:53, 140.11it/s]
24149it [02:53, 140.07it/s]
24164it [02:53, 139.46it/s]
24179it [02:53, 139.59it/s]
24193it [02:53, 138.46it/s]
24207it [02:53, 138.

31535it [03:45, 141.07it/s]
31550it [03:46, 139.33it/s]
31565it [03:46, 140.99it/s]
31580it [03:46, 142.35it/s]
31595it [03:46, 143.81it/s]
31610it [03:46, 143.50it/s]
31625it [03:46, 142.20it/s]
31640it [03:46, 141.99it/s]
31655it [03:46, 141.92it/s]
31670it [03:46, 131.04it/s]
31684it [03:46, 132.23it/s]
31699it [03:47, 136.12it/s]
31713it [03:47, 135.85it/s]
31728it [03:47, 138.62it/s]
31743it [03:47, 140.48it/s]
31758it [03:47, 140.65it/s]
31773it [03:47, 140.56it/s]
31788it [03:47, 141.38it/s]
31803it [03:47, 142.32it/s]
31818it [03:47, 143.18it/s]
31833it [03:48, 142.46it/s]
31848it [03:48, 142.98it/s]
31863it [03:48, 142.63it/s]
31878it [03:48, 142.97it/s]
31893it [03:48, 143.05it/s]
31908it [03:48, 143.03it/s]
31923it [03:48, 143.22it/s]
31938it [03:48, 142.73it/s]
31953it [03:48, 143.49it/s]
31968it [03:48, 143.96it/s]
31983it [03:49, 144.28it/s]
31998it [03:49, 144.82it/s]
32013it [03:49, 143.72it/s]
32028it [03:49, 141.81it/s]
32043it [03:49, 141.56it/s]
32058it [03:49, 141.

39371it [04:41, 144.47it/s]
39386it [04:41, 144.93it/s]
39401it [04:41, 145.46it/s]
39416it [04:41, 144.70it/s]
39431it [04:41, 144.77it/s]
39446it [04:42, 144.68it/s]
39462it [04:42, 146.22it/s]
39477it [04:42, 145.27it/s]
39492it [04:42, 145.08it/s]
39507it [04:42, 144.24it/s]
39522it [04:42, 143.75it/s]
39537it [04:42, 142.92it/s]
39552it [04:42, 141.22it/s]
39567it [04:42, 139.47it/s]
39581it [04:42, 139.07it/s]
39596it [04:43, 139.41it/s]
39610it [04:43, 139.25it/s]
39625it [04:43, 141.16it/s]
39640it [04:43, 139.10it/s]
39654it [04:43, 138.67it/s]
39668it [04:43, 138.92it/s]
39683it [04:43, 140.07it/s]
39698it [04:43, 140.70it/s]
39713it [04:43, 141.94it/s]
39728it [04:44, 141.99it/s]
39743it [04:44, 142.23it/s]
39758it [04:44, 143.86it/s]
39773it [04:44, 143.78it/s]
39788it [04:44, 143.04it/s]
39803it [04:44, 143.80it/s]
39818it [04:44, 144.83it/s]
39833it [04:44, 144.40it/s]
39848it [04:44, 144.08it/s]
39863it [04:44, 144.04it/s]
39878it [04:45, 144.24it/s]
39893it [04:45, 142.

47063it [05:36, 132.27it/s]
47077it [05:37, 133.15it/s]
47091it [05:37, 134.95it/s]
47106it [05:37, 136.26it/s]
47120it [05:37, 136.54it/s]
47134it [05:37, 136.51it/s]
47148it [05:37, 135.85it/s]
47162it [05:37, 136.78it/s]
47176it [05:37, 136.24it/s]
47190it [05:37, 135.45it/s]
47204it [05:37, 134.71it/s]
47218it [05:38, 133.75it/s]
47232it [05:38, 133.66it/s]
47246it [05:38, 133.01it/s]
47260it [05:38, 132.24it/s]
47274it [05:38, 132.92it/s]
47288it [05:38, 132.95it/s]
47302it [05:38, 133.86it/s]
47316it [05:38, 134.84it/s]
47330it [05:38, 135.21it/s]
47344it [05:39, 136.13it/s]
47358it [05:39, 135.48it/s]
47372it [05:39, 135.54it/s]
47386it [05:39, 135.46it/s]
47400it [05:39, 134.68it/s]
47414it [05:39, 134.99it/s]
47428it [05:39, 134.65it/s]
47442it [05:39, 134.85it/s]
47456it [05:39, 134.35it/s]
47470it [05:39, 133.88it/s]
47484it [05:40, 132.18it/s]
47498it [05:40, 121.78it/s]
47512it [05:40, 125.01it/s]
47526it [05:40, 127.66it/s]
47540it [05:40, 130.71it/s]
47555it [05:40, 133.

In [ ]:
anno_peaks.to_csv('Results/peaks_with_tss.tsv',sep='\t')

In [20]:
anno_peaks = pd.read_csv('Results/peaks_with_tss.tsv',sep='\t',index_col=0)
anno_peaks = anno_peaks[['Chr','Start','End','Strand','Peak Score','Nearest TSS','isSameStrand','Distance to TSS']]
anno_peaks = anno_peaks[~anno_peaks['Nearest TSS'].isnull()]

### Do the same but for genes

In [ ]:
%%time
txn_to_gene = dict()
txns = set(tss['transcript_id'])
for tx in tqdm_notebook(txns):
    txn_to_gene[tx] = tss[tss['transcript_id'] == tx]['gene'][0]
    


In [15]:
for key in txn_to_gene:
    txn_to_gene[key] = txn_to_gene[key][0]

In [17]:
pickle.dump(txn_to_gene,open('Results/txn_to_gene','wb'))

In [19]:
txn_to_gene

{'asmbl_22997': 'CNBP',
 'asmbl_50212': 'VN1R4',
 'asmbl_22993': 'COPG1',
 'asmbl_22992': 'HMCES',
 'asmbl_22991': 'HMCES',
 'asmbl_22990': 'HMCES',
 'asmbl_50215': 'ZNF416',
 'asmbl_22998': 'CNBP',
 'asmbl_720': 'CACNB3',
 'asmbl_721': 'CACNB3',
 'asmbl_27819': 'CML2',
 'asmbl_27818': 'CML6',
 'asmbl_27811': 'PRADC1',
 'asmbl_27810': 'PRADC1',
 'asmbl_27813': 'CCT7',
 'asmbl_27812': 'PRADC1',
 'asmbl_27815': 'FBXO41',
 'asmbl_27817': 'EGR4',
 'asmbl_27816': 'FBXO41',
 'asmbl_16341': 'DNAAF4',
 'asmbl_7681': 'MCMBP',
 'asmbl_62869': 'MEMO1',
 'asmbl_62862': 'CEP85L',
 'asmbl_62861': 'MCM9',
 'asmbl_62860': 'MCM9',
 'asmbl_76196': 'SSR1',
 'asmbl_76197': 'SSR1',
 'asmbl_76194': 'SSR1',
 'asmbl_40945': 'REST',
 'asmbl_76195': 'SSR1',
 'asmbl_64169': 'EYA2',
 'asmbl_77929': 'PELP1',
 'asmbl_64164': 'ZMYND8',
 'asmbl_76192': 'RIOK1',
 'asmbl_64166': 'ZMYND8',
 'asmbl_64161': 'ZMYND8',
 'asmbl_64162': 'ZMYND8',
 'asmbl_18893': 'GIMAP7',
 'asmbl_18892': 'GIMAP7',
 'asmbl_18890': 'GIMAP1',
 '

In [21]:
anno_peaks

,Chr,Start,End,Strand,Peak Score,Nearest TSS,isSameStrand,Distance to TSS
PeakID (cmd=annotatePeaks.pl Results/merged_samples/duplicatesSeparate_peaks_merged.tsv /data/genome/hamster/picr/picr.fa -gff /data/genome/hamster/picr/updated_final_sort.gff3),,,,,,,,
Merged-picr_33-4827738-24,picr_33,4827637,4827815,+,277526.625000,asmbl_45526,True,4.0
Merged-picr_117-401925-25,picr_117,401850,402000,-,225524.000000,asmbl_11728,True,-147.0
Merged-picr_35-689698-12,picr_35,689623,689773,-,199731.640625,asmbl_46769,False,1404.0
Merged-picr_76-305376-6,picr_76,305301,305451,-,184552.890625,asmbl_75726,False,-23522.0
Merged-picr_54-9283439-16,picr_54,9283364,9283514,+,146467.000000,asmbl_62508,False,-871.0
Merged-picr_5-12595651-4,picr_5,12595575,12595732,-,129681.921875,asmbl_58436,False,-163973.0
Merged-picr_146-1769376-10,picr_146,1769295,1769461,+,108833.570312,asmbl_21334,False,-16259.0
Merged-picr_102-1919429-10,picr_102,1919354,1919504,+,96292.867188,asmbl_7500,True,1499.0
Merged-picr_54-1311935-23,picr_54,1311860,1312010,+,83866.054688,asmbl_62111,True,69544.0


In [26]:
%%time
anno_peaks['Nearest gene'] = ''

for ind,val in tqdm_notebook(anno_peaks.iterrows()):
    if val['Nearest TSS'] in txn_to_gene:
        anno_peaks.set_value(ind,'Nearest gene',txn_to_gene[val['Nearest TSS']])

7328it [00:16, 452.85it/s]

CPU times: user 1min 38s, sys: 15.8 s, total: 1min 54s
Wall time: 1min 38s


In [27]:
anno_peaks.to_csv('Results/peaks_with_tss_and_genes.tsv',sep='\t')

In [28]:
anno_peaks.head()

,Chr,Start,End,Strand,Peak Score,Nearest TSS,isSameStrand,Distance to TSS,Nearest gene
PeakID (cmd=annotatePeaks.pl Results/merged_samples/duplicatesSeparate_peaks_merged.tsv /data/genome/hamster/picr/picr.fa -gff /data/genome/hamster/picr/updated_final_sort.gff3),,,,,,,,,
Merged-picr_33-4827738-24,picr_33,4827637,4827815,+,277526.625000,asmbl_45526,True,4.0,CPA2
Merged-picr_117-401925-25,picr_117,401850,402000,-,225524.000000,asmbl_11728,True,-147.0,ATN1
Merged-picr_35-689698-12,picr_35,689623,689773,-,199731.640625,asmbl_46769,False,1404.0,SLC3A2
Merged-picr_76-305376-6,picr_76,305301,305451,-,184552.890625,asmbl_75726,False,-23522.0,OR1S1
Merged-picr_54-9283439-16,picr_54,9283364,9283514,+,146467.000000,asmbl_62508,False,-871.0,SURF1


In [29]:
anno_peaks = pd.read_csv('Results/peaks_with_tss_and_genes.tsv',sep='\t',index_col=0)

### Overlap gene_list to SecM

### How many peaks for a specific gene. 

In [32]:
gene_pd.loc[curr_gene,'Peaks'] 

[]

In [71]:
distance = [-1000,100]
(9773.0 > distance[0]) & (9773 < distance[1])

False

In [66]:
distance[1]

100

In [67]:
9773.0 < distance[1]

False

In [72]:
def get_genes_peaks(peaks,gene,distance=[-1000,+100]):
    filt = peaks[(peaks['Nearest gene'] == gene) & 
                 ((peaks['Distance to TSS'] > distance[0]) & (peaks['Distance to TSS'] < distance[1]))]
    return filt

def get_nearest_tss(peaks_gene): 
    #peaks of one gene
    peaks_gene['abs'] = np.abs(peaks_gene['Distance to TSS'])
    idx = peaks_gene['abs'].argmin()
    return idx, peaks_gene.loc[idx,'Distance to TSS']


def tss_per_gene(peaks_gene): 
    #peaks of one gene
    return len(set(peaks_gene['Nearest TSS']))


def tissues_gene_expressed(peaks_gene,peaks_tissue_matrix):
    #peaks of one gene
    curr_gene_vals = peaks_tissue_matrix[peaks_tissue_matrix.isin(peaks_gene.idnex)]
    tiss = (curr_gene_vals > 1).any(axis=1)
    tiss = tiss.columns[tiss]
    return tiss

In [55]:
curr

,Chr,Start,End,Strand,Peak Score,Nearest TSS,isSameStrand,Distance to TSS,Nearest gene
PeakID (cmd=annotatePeaks.pl Results/merged_samples/duplicatesSeparate_peaks_merged.tsv /data/genome/hamster/picr/picr.fa -gff /data/genome/hamster/picr/updated_final_sort.gff3),,,,,,,,,
Merged-picr_10-37581004-2,picr_10,37580929,37581079,-,76.650002,asmbl_6549,True,8130.0,110KD_PLAKN
Merged-picr_10-37582647-1,picr_10,37582572,37582722,-,56.099998,asmbl_6549,True,9773.0,110KD_PLAKN
Merged-picr_10-37579922-1,picr_10,37579847,37579997,-,53.799999,asmbl_6549,True,7048.0,110KD_PLAKN
Merged-picr_10-37568061-2,picr_10,37567979,37568143,+,48.050003,asmbl_6549,False,-4813.0,110KD_PLAKN
Merged-picr_10-37578310-2,picr_10,37578185,37578435,+,45.299999,asmbl_6549,False,5436.0,110KD_PLAKN
Merged-picr_10-37578195-3,picr_10,37578100,37578312,-,44.266666,asmbl_6549,True,5332.0,110KD_PLAKN
Merged-picr_10-37583874-2,picr_10,37583789,37583959,-,41.150002,asmbl_6549,True,11000.0,110KD_PLAKN
Merged-picr_10-37579833-5,picr_10,37579735,37579957,+,39.664001,asmbl_6549,False,6972.0,110KD_PLAKN
Merged-picr_10-37583986-3,picr_10,37583893,37584076,+,33.133335,asmbl_6549,False,11110.0,110KD_PLAKN


In [45]:
gene_pd.loc['110KD_PLAKN']

Peaks                       [Merged-picr_10-37581004-2, Merged-picr_10-375...
Number of Peaks                                                            28
Tissues                                                                   NaN
minDistance                                                               -22
minDistancePeak                                     Merged-picr_10-37572852-1
maxValue                                                                  NaN
Peaks within -1kb:+100bp                                                  NaN
Name: 110KD_PLAKN, dtype: object

In [52]:
gene_pd.loc['110KD_PLAKN']

Peaks                       [Merged-picr_10-37581004-2, Merged-picr_10-375...
Number of Peaks                                                            28
Tissues                                                                   NaN
minDistance                                                               -22
sameStrandPeaks             [Merged-picr_10-37581004-2, Merged-picr_10-375...
minDistancePeak                                     Merged-picr_10-37572852-1
maxValue                                                                  NaN
Peaks within -1kb:+100bp                                                  NaN
Name: 110KD_PLAKN, dtype: object

In [ ]:
gene_list = tss['gene'].unique()
gene_pd = pd.DataFrame(index=gene_list,columns=['Peaks','Number of Peaks','Tissues','minDistance',
                                                'sameStrandPeaks','minDistancePeak','maxValue','Peaks within -1kb:+100bp'])
gene_pd['Number of Peaks'] = 0
gene_pd['Peaks'] = ''
gene_pd['minDistance'] = 0
gene_pd['minDistancePeak'] = ''
#gene_pd['Peaks'] = [[] for _ in range(gene_pd.shape[0])]

for i in tqdm_notebook(anno_peaks.groupby(['Nearest gene'])):
    curr_gene = i[0]
    #curr_gene = tss[tss['transcript_id']==i[0]]['gene']
    gene_pd.loc[curr_gene,'Number of Peaks'] += len(i[1])
    #curr = (gene_pd.loc[curr_gene,'Peaks'])
    curr = i[1]
    #gene_pd.loc[curr_gene,'Peaks'] = curr #.append(list(i[1].index))
    gene_pd.set_value(curr_gene,'Peaks',list(curr.index))
    
    #print('before',curr)
    ## Filter to only within the distance of -1kb to +100bp
    curr = get_genes_peaks(curr,curr_gene)
    gene_pd.set_value(curr_gene,'Peaks within -1kb:+100bp',list(curr.index))
    
    ## isSameStrand 
    same_strand = curr[curr['isSameStrand'] == True]
    gene_pd.set_value(curr_gene,'sameStrandPeaks',list(same_strand.index))
    ## Find closest peak
    if len(same_strand) > 0:
        minPeak, minPeakValue = get_nearest_tss(same_strand)
        gene_pd.set_value(curr_gene,'minDistance',minPeakValue)
        gene_pd.set_value(curr_gene,'minDistancePeak',minPeak)
    
    ## Peaks
    
    ## Strongest peak
    

/home/isshamie/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### How many peaks for a specific gene. 
### How many tissues 
### What was the strongest peak
### Distance between peaks (if we separate the transcripts out)
### How many start sites


In [ ]:
def peaks_from_group():

In [ ]:
#sns.distplot(start_sites_gff['Experimental'],kde=False,norm_hist=False)

f = plt.figure();
ax = f.add_subplot(1,1,1);
ax.hist(tss['Experimental']);

ax.set_xlabel('Number of peaks per TSS')

plt.savefig('Results/Figures/Number_of_peaks_per_gene.png')

## Check if any peaks had a nearest TSS that was not in the tss file 
#### (this would happen b/c the TSSs were collapsed to take only one transcript_id if they shared the same start site, taking the most confident one)

In [ ]:
for ind,val in anno_peaks.iterrows():
    

In [8]:
tss['Experimental'] = 0
tss['Peaks'] = np.nan
tss['Peaks'] = tss['Peaks'].astype(object)
for ind,val in tqdm.tqdm(tss.iterrows()):
    curr_peaks = promoter_peaks[promoter_peaks['Annotation'].str.contains(val['ID'] + '\.')] #\. is for not count asmbl_21 for asmbl_2 
    if not len(curr_peaks) == 0:
        tss.set_value(ind,'Experimental',len(curr_peaks))
        tss.set_value(ind,'Peaks',list(curr_peaks.index.values))
    


31560it [1:04:25,  8.34it/s]

KeyboardInterrupt: 